# Images

A **Docker image** is a read-only template that contains a set of instructions for creating a **container** that can run on the **Docker** platform. It provides a convenient way to package up applications **and** preconfigured server environments, which you can use for your own private use or share publicly with other **Docker** users.

Here is the summary.

- Layers

- Dockerfile

- Build an image

- Multistage

- Cache memory

- Build context

- Deploy/share an image

- Useful options

## Layers

Docker containers are building blocks for applications. Each container is an image with a readable/writeable layer on top of a bunch of read-only layers.

For example, here is a Dockerfile for creating a node js web app image. It shows the commands that are executed to create the image.

In [ ]:
%%bash
FROM node:latest
# Create app directory
RUN mkdir -p /usr/src/app
WORKDIR /usr/src/app
# Install app dependencies
COPY package.json /usr/src/app/
RUN npm install
# Bundle app source
COPY . /usr/src/app
EXPOSE 8080
CMD [ "npm", "start" ]

Each order creates a new layer. When an image is instantiated into a container, Docker adds an additional read and write layer to allow the application to make changes in the file system.

<img src="file:///home/pth/Documents/malt/LEAD_PROGRAM/M04-DevOps/D03-Standardize_your_environment_with_Docker/00-Lectures/container-layers.jpg" title="" alt="container-layers.jpg" data-align="center">

## Dockerfile

Docker can build images by reading the instructions from a `Dockerfile`. A `Dockerfile` is a text document that contains all the commands a user could call on the command line to assemble an image. Using `docker build` users can create an automated build that executes several command-line instructions in succession.

Here is the list of instructions for a Dockerfile :

In [ ]:
FROM # To choose the image on which to base your choice, always go first.
RUN # Executes a command
CMD # Command executed when the default container is started.
EXPOSE # Opens a port
ENV # Allows you to edit environment variables
ARG # Sort of like ENV, but only for the time it takes to build up the image.
COPY # Copies a file or directory from the host to the image.
ADD # Allows you to copy a file from the host or from a URL to the image, also allows you to decompress a tar archive.
LABEL # Useful metadata for certain container management software, such as rancher or swarm, or simply to put information on the image.
ENTRYPOINT # Command executed at container startup, cannot be modified, used to package a command.
VOLUME # Creates a specific partition
WORKDIR # Allows you to choose the working directory
USER # Selects the user at the start of the ENTRYPOINT or CMD command.
ONBUILD # Creates a step that will be executed only if our image is chosen as the base.
HEALTHCHECK # Allows you to add a command to check the operation of your container
STOPSIGNAL # Selects the [signal](http://man7.org/linux/man-pages/man7/signal.7.html) that will be sent to the container when the container is empty.

In the following, we will put these intructions into practice.

## Build an image

The dockerfile is a recipe for images, using this command, your computer will follow the recipe to create your custom image.

In [ ]:
%%bash
docker image build -t [imagename]:[tag] [dockerfile folder]

The tag makes it possible to distinguish the versions of an image. *0.1* or *beta* are valid tags.

Following an example together easy.

In [ ]:
FROM alpine:latest
RUN apk update
RUN apk add python3

1. We base the image on a very light distribution of linux : *alpine*

2. Update the index of available packages

3. Install python3

In [ ]:
%%bash
docker build -t alpinepython:alpha .

1. the name of the image will be "alpinepython"

2. The **.** means that the dockerfile is present in the current folder.

Let's test the creation of our image.

In [ ]:
%%bash
docker container run -it alpinepython:alpha python3

1. The image is launched using its name and tag.

2. We launch *python3* when the container is started. 

3. Request to open an interactive terminal 

Here are the results :

In [ ]:
Python 3.8.5 (default, Jul 20 2020, 23:11:29) 
[GCC 9.3.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> 2+2
4

You are ready to do exercise 3.

## Multistage

In this section we will see how to optimise our docker images. We will learn how to create smaller images by separating the build environment from the runtime environment.

To demonstrate the difference between the build environment and the runtime environment, we will use a hello world program written in Java. The following command create `HelloWorld.java` which we will use throughout this tutorial:

In [ ]:
%%bash
cat >HelloWorld.java <<EOF
class HelloWorld {
    public static void main(String[] a) {
        System.out.println("Hello world!");
    }
}
EOF

The straight forward way of building and packaging the Java program is by using the Java Development Kit (JDK). Consider the following `Dockerfile`:

In [ ]:
%%bash
cat >Dockerfile <<EOF
FROM openjdk:11-jdk
COPY HelloWorld.java .
RUN javac HelloWorld.java
CMD java HelloWorld
EOF

When building it, the resulting image will contain the JDK, the source code and the compiled hello world program:

In [ ]:
%%bash
docker image build --tag helloworld:huge .

Although it produces the correct output…

In [ ]:
%%bash
docker container run helloworld:huge

It's way too large for running the compiled program that only requires the Java Runtime Environment (JRE), which is much smaller then the JDK. In addition, it is not necessary to ship the source code to execute the program.

In the end, the resulting image will be more than 600MB large:

In [ ]:
%%bash
docker image ls;
REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
helloworld          huge                a2742b8a9477        10 seconds ago      627MB

Separating the build environment allows for much smaller images because the runtime environment usually required much less space. We can combine the approach of building the program using the JDK and packaging it using the JRE in a single `Dockerfile` using the multi-stage build feature.

Each stage begins with a `FROM` statement and is named using the `AS` keyword. When building such a multi-stage `Dockerfile`, only the last stage describes the final image and its contents.

The following `Dockerfile` describes a multi-stage build for the hello world program used above:

In [ ]:
%%bash
cat >Dockerfile <<EOF
FROM openjdk:11-jdk AS build
COPY HelloWorld.java .
RUN javac HelloWorld.java

FROM openjdk:11-jre AS run
COPY --from=build HelloWorld.class .
CMD java HelloWorld
EOF

Using the above example, building the Java program is performed in the first stage using the JDK image. The second (and last) stage is based on the JRE image and copies the compiled program from the first stage. Note the new syntax for the `COPY` statement where it takes a parameter called `--from` specifying the build stage to copy from.

Building the image looks very similar to the well-known process:

In [ ]:
%%bash
docker image build --tag helloworld:small .

The resulting image will work as expected…

In [ ]:
%%bash
docker container run helloworld:small

…but have the same small size as above:

In [ ]:
%%bash
docker image ls

## Cache memory

When writing a dockerfile, the cache mechanism must be taken into account to optimise the image construction time. Between two constructions the cache can be used to save time. 
The cache can be used to rebuild an image after changing a configuration file. If the cache is used correctly, docker will not need to recompile the source code of the application.
There are several ways to force layers to be built if needed. When you change a layer (an ADD / COPY statement) all the top layers will be invalidated and rebuilt.

We will build an image for a hello world node application. Here is the dockerfile.

In [ ]:
FROM node:10.18.1
WORKDIR /app
COPY . .
RUN npm i
EXPOSE 80
CMD ["node", "app.js"]

Here is the source code of the application and the .json package.

package.json :

In [ ]:
{
  "name": "nodeapp",
  "version": "1.0.0",
  "description": "",
  "main": "index.js",
  "scripts": {
    "test": "echo \"Error: no test specified\" && exit 1"
  },
  "keywords": [],
  "author": "",
  "license": "ISC"
}

app.js :

In [ ]:
console.log("Hello world!");

During the first build of our image, docker builds the layers from scratch.

In [ ]:
%%bash
$ docker build -t nodeapp:1.0 .
Sending build context to Docker daemon   5.12kB
Step 1/6 : FROM node:10.18.1
 ---> 4c447b45159a
Step 2/6 : WORKDIR /app
 ---> Using cache
 ---> a3c7fb356a92
Step 3/6 : COPY . .
 ---> eff5568ac2cc
Step 4/6 : RUN npm i
 ---> Running in 6c3884421a6d
Removing intermediate container 6c3884421a6d
 ---> 61990a5ae4c7
Step 5/6 : EXPOSE 80
 ---> Running in 36b7a1373e8e
Removing intermediate container 36b7a1373e8e
 ---> 0ac34980dbdb
Step 6/6 : CMD ["node", "app.js"]
 ---> Running in 8c669631004b
Removing intermediate container 8c669631004b
 ---> 2c738077c79d
Successfully built 2c738077c79d
Successfully tagged nodeapp:1.0

Let's run the same command again to see what happens.

In [ ]:
%%bash
docker build -t nodeapp:1.0 .
Sending build context to Docker daemon   5.12kB
Step 1/6 : FROM node:10.18.1
 ---> 4c447b45159a
Step 2/6 : WORKDIR /app
 ---> Using cache
 ---> a3c7fb356a92
Step 3/6 : COPY . .
 ---> Using cache
 ---> eff5568ac2cc
Step 4/6 : RUN npm i
 ---> Using cache
 ---> 61990a5ae4c7
Step 5/6 : EXPOSE 80
 ---> Using cache
 ---> 0ac34980dbdb
Step 6/6 : CMD ["node", "app.js"]
 ---> Using cache
 ---> 2c738077c79d
Successfully built 2c738077c79d
Successfully tagged nodeapp:1.0

The construction is very fast and you can see the note "---> Using cache".
Docker notices that the layers have not changed because all the instructions remain the same.

Make a modification in the source code of the application. Here is the new app.js file.

In [ ]:
%%bash
console.log("Docker is great.");

In [ ]:
%%bash
docker build -t nodeapp:1.0 .
Sending build context to Docker daemon   5.12kB
Step 1/7 : FROM node:10.18.1
 ---> 4c447b45159a
Step 2/7 : WORKDIR /app
 ---> Using cache
 ---> a3c7fb356a92
Step 3/7 : COPY . .
 ---> 1160eaef9d15
Step 5/7 : RUN npm i
 ---> Running in 4a34a3d2e664
Removing intermediate container 4a34a3d2e664
 ---> b0728166a40e
Step 6/7 : EXPOSE 80
 ---> Running in f99a4501f765
Removing intermediate container f99a4501f765
 ---> acf04ce11e86
Step 7/7 : CMD ["node", "app.js"]
 ---> Running in 28bb7d7fc2fc
Removing intermediate container 28bb7d7fc2fc
 ---> 7f737d7b6151
Successfully built 7f737d7b6151
Successfully tagged nodeapp:1.0

The daemon docker detects changes in the source code. It invalidated the cache in the third step. Following layers were reconfigured. 

In our example, this is not very impacting as we have no dependencies. In an application with a lot of dependencies, building the image can take a lot of time. 
We don't want all the dependencies to be recovered and rebuilt when changing single line of code.
To solve this problem we will modify the Dockerfile. We're going to **split** the "COPY" instruction. In the first copy we only take the "package.json" and install the dependencies. We then copy the source code of the application.
Here the new Dockerfile :

In [ ]:
FROM node:10.18.1
WORKDIR /app
COPY package.json .
RUN npm i
COPY app.js .
EXPOSE 80
CMD ["node", "app.js"]

When modifying a line of code, the cache after the step 5 will change.

In [ ]:
%%bash
docker build -t nodeapp:1.0 .
Sending build context to Docker daemon   5.12kB
Step 1/7 : FROM node:10.18.1
 ---> 4c447b45159a
Step 2/7 : WORKDIR /app
 ---> Using cache
 ---> a3c7fb356a92
Step 3/7 : COPY package.json .
 ---> Using cache
 ---> 0d6631054baa
Step 4/7 : RUN npm i
 ---> Using cache
 ---> 3590f4664c3f
Step 5/7 : COPY app.js .
 ---> c776e365a5d3
Step 6/7 : EXPOSE 80
 ---> Running in c8e3ed8ec07b
Removing intermediate container c8e3ed8ec07b
 ---> 453fc69bad63
Step 7/7 : CMD ["node", "app.js"]
 ---> Running in 9630e5c56e1a
Removing intermediate container 9630e5c56e1a
 ---> f9dc7bf1b56b
Successfully built f9dc7bf1b56b
Successfully tagged nodeapp:1.0

Dependencies are only rebuilt after each change in the code.

## Build context

The build context is the set of files located at the specified PATH or URL. Those files are sent to the Docker daemon during the build so it can use them in the filesystem of the image.

Usually, we use a command like the following one to build the image, the Dockerfile being at the root of the project’s folder:

In [ ]:
%%bash
docker build -t nodeapp:1.0 .

In that case, the build context is the content of the current folder (“.” specified as the last element of the command).

You have to be sure that the build context only contains the files and folders it really needs. This is where the dockerignore file comes into play It is the same as with the gitignore file which allows not to send sensitive data on github.

Before the docker CLI sends the context to the docker daemon, it looks for a file named `.dockerignore` in the root directory of the context. If this file exists, the CLI modifies the context to exclude files and directories that match patterns in it. This helps to avoid unnecessarily sending large or sensitive files and directories to the daemon and potentially adding them to images using `ADD` or `COPY`. 

Many projects are managed with the git version manager. Git creates a .git folder at the root. In this context it is not useful that this file is in the image. 

 Start handling the nodeapp project with Git.

In [ ]:
%%bash
git init;

Create the image:

In [ ]:
%%bash
docker build -t nodeapp:1.0 .
Sending build context to Docker daemon   42.5kB
Step 1/7 : FROM node:10.18.1
 ---> 4c447b45159a
Step 2/7 : WORKDIR /app
[...]

We see that the context sent to the demon docker has changed from 5.12kb to 42.5kB.

Let's create a .dockerignore file:

In [ ]:
%%bash
echo ".git" > .dockerignore

And build the app:

In [ ]:
%%bash
docker build -t nodeapp:1.0 .
Sending build context to Docker daemon  6.144kB
Step 1/7 : FROM node:10.18.1
 ---> 4c447b45159a
Step 2/7 : WORKDIR /app
[...]

The git file is no longer sent to the docker deamon. The build context size is reduced by 35kb.

## Deploy/share an image

We can share an image by sending it to the hub.

To get started, you need to create an account on the Hub, go to [Docker Hub](https://hub.docker.com/).

Once registered, you log in from our docker :

In [ ]:
%%bash
docker login;
Login with your Docker ID to push and pull images from Docker Hub. If you don't have a Docker ID, head over to https://hub.docker.com to create one.
Username: 
Password: 
WARNING! Your password will be stored unencrypted in /home/pth/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded

Now that we are logged in, we can push our image, to do so, your image must be named in this form: username/imagename:tag :

In [ ]:
%%bash
docker image push username/nodeapp:1.0;

You can go to your Hub, and you will find a new repository.

## Useful commands

I invite you to read the official docker documentation and browse through all the possible commands related [here]([docker image | Docker Documentation](https://docs.docker.com/engine/reference/commandline/image/)